# Batch Analysis for hawk/dove simulation with multiple risk attitudes, no adjustment

- What % of time do agents play Hawk, by risk attitude? risk-inclined (R=1) and risk-avoidant (R8) agents play Hawk?
- Cumulative wealth analysis by risk attitude

To track how often agents play Hawk, we need to collect data for every round.

This data was generated by running:
```console
simulatingrisk/hawkdovemulti/batch_run.py --params no_adjustment --agent-data --collect-data every_round
```

Each row in the data file represents one round for each agent.

In [23]:
import pandas as pd

df = pd.read_csv("../../data/hawkdovemulti/2025-07-22T170057_747737_agent.csv")
# code still uses risk_level internally, but relabel as risk attitude
df = df.rename(columns={'risk_level': 'risk_attitude'})
# drop the risk_level_changed, since it is not relevant here (no adjustment = no changes)
df = df.drop("risk_level_changed", axis=1)
# add a numeric field to turn choice of play to 1/0 hawk, for aggregation
df['played_hawk'] = df.choice.apply(lambda x: 1 if x == 'hawk' else 0)
df.head()

,RunId,iteration,Step,AgentID,risk_attitude,choice,points,played_hawk
0,0,0,1,0,7,hawk,18,1
1,0,0,1,1,9,dove,14,0
2,0,0,1,2,1,hawk,12,1
3,0,0,1,3,2,dove,12,0
4,0,0,1,4,8,dove,11,0


## Percent of the time agents play Hawk, by risk attitude

What % of time do risk-inclined (R=1) and risk-avoidant (R8) agents play Hawk?

- Guess from observation is is >90% for R=1, <10% for R8, but we want to have statistics for this: for X trials, how many of them does R1 play Hawk more than 90% of the time?
- Also useful to have statistics e.g. R=2 played Hawk between 80-90% of the time, or whatever the result is.


In [63]:
# each row in the data frame is a play by an agent on the grid
# group by risk level, then:
# - count the number of rows 
# - sum the played_hawk field (= number of times played hawk)

hawk_by_risk_attitude = df.groupby("risk_attitude", as_index=False)["played_hawk"].agg(["count", "sum"])

# use aggregate values to calculate % of time agents by risk attitude played hawk based on the count and sum
hawk_by_risk_attitude["pct_play_hawk"] = hawk_by_risk_attitude.apply(lambda x: (x["sum"] / x["count"]) * 100, axis=1)

hawk_by_risk_attitude

,risk_attitude,count,sum,pct_play_hawk
0,0,355053,350581,98.740470
1,1,362889,349522,96.316504
2,2,356945,341464,95.662917
3,3,361025,301738,83.578146
4,4,372208,253542,68.118364
5,5,359406,113174,31.489179
6,6,363046,58901,16.224115
7,7,360965,15092,4.181015
8,8,353243,12331,3.490798
9,9,353820,4497,1.270985


In [90]:
(hawk_by_risk_attitude[["risk_attitude", "pct_play_hawk"]]).style \
  .format(precision=1, thousands=".", decimal=".") \
  .relabel_index(["Risk Attitude", "% Hawk"], axis=1) \
  .set_caption("% of time agents play Hawk") \
  .format(lambda x: f"{x:.1f}%", subset='pct_play_hawk') \
  .hide()   # hide the index



Risk Attitude,% Hawk
0,98.7%
1,96.3%
2,95.7%
3,83.6%
4,68.1%
5,31.5%
6,16.2%
7,4.2%
8,3.5%
9,1.3%


In [127]:
import altair as alt

alt.Chart(hawk_by_risk_attitude).mark_bar(width=10).encode(
    x=alt.X("risk_attitude", title="Risk Attitude").scale(domain=[0, 9]),                                                          
    y=alt.Y("pct_play_hawk", title="% time plays Hawk")
).properties(title="% of time agents play Hawk")

alt.Chart(...)

Context: how many runs are these numbers drawn from?



In [109]:
len(df.RunId.unique())

total_unique_runs = len(df.RunId.unique())
total_iterations = len(df.iteration.unique())
n_combinations = int(total_unique_runs / total_iterations)

# Step is the round count indicator
longest_run = df.Step.max()    # highest across all runs
# average of max value for each run
average_run = df.groupby('RunId')['Step'].max().mean()

print(f"""{total_unique_runs} total unique runs; {total_iterations} iterations of {n_combinations} different parameter combinations.

Longest run: {longest_run} steps
Average run: {average_run:.1f} steps
""")

900 total unique runs; 100 iterations of 9 different parameter combinations.

Longest run: 68 steps
Average run: 40.0 steps



In [ ]:
# possible further refinement/analysis: do these numbers vary based on starting parameters?
# ... requires pulling in associated model data to match run ids with starting parameters
# should be possible to group twice https://stackoverflow.com/questions/45561118/how-i-can-apply-groupby-two-times-on-pandas-data-frame
# then maybe we can do a facet chart to compare results across parameters

## Cumulative Wealth analysis


- mean and quartiles for wealth by R
- does mean vary between Rs or is it roughly the same?
  - quartiles look different, but need a statistic; esp. compare lower-R quartile to higher-R quartile
     - expect/hope that lower quartile is higher for R1 than R8, higher quartile is higher for R8 than R1
     - what's going on in the middle?


In [131]:
# for wealth analysis, we only want to look at the last round (Step) of each run

last_round_df = df.groupby('RunId', as_index=False)['Step'].max()
last_round_df.head()

,RunId,Step
0,0,31
1,1,31
2,2,62
3,3,62
4,4,31


In [160]:
# combine the last round dataframe and full agent dataframe to get just the last round
agents_last_round_df = pd.merge(last_round_df, df, on=['RunId', 'Step'], how="left")

# hard to compare points, since it depends on how long the simulation ran; can we scale?
agents_last_round_df['scaled_points'] = agents_last_round_df.apply(lambda x: (x['points'] / x['Step'])*10, axis=1)

agents_last_round_df.head(10)

,RunId,Step,iteration,AgentID,risk_attitude,choice,points,played_hawk,scaled_points
0,0,31,0,0,7,dove,334,0,107.741935
1,0,31,0,1,9,dove,374,0,120.645161
2,0,31,0,2,1,hawk,285,1,91.935484
3,0,31,0,3,2,hawk,282,1,90.967742
4,0,31,0,4,8,dove,372,0,120.000000
5,0,31,0,5,7,dove,343,0,110.645161
6,0,31,0,6,0,hawk,507,1,163.548387
7,0,31,0,7,7,dove,343,0,110.645161
8,0,31,0,8,3,hawk,468,1,150.967742
9,0,31,0,9,8,dove,313,0,100.967742


In [153]:

# our data has a lot of rows; enable vegafusion so altair can calculate quartilies for us
alt.data_transformers.enable("vegafusion")

alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
    x="risk_attitude",
    y="points"
)

alt.Chart(...)

In [163]:
alt.Chart(agents_last_round_df).mark_boxplot(extent="min-max").encode(
    x="risk_attitude",
    y="scaled_points"
).properties(title="Cumulative wealth, scaled by simulation length")

alt.Chart(...)

In [172]:
# what if we look at wealth at round 31 across simulations?

agents_round31_df = df[df['Step'] == 31]

alt.Chart(agents_round31_df).mark_boxplot(extent="min-max").encode(
    x="risk_attitude",
    y="points"
).properties(title="Cumulative wealth at round 31 across runs")

alt.Chart(...)

In [151]:
# Altair boxplot is calculating quartiles for us, but we can calculate them directly as well

# Q1: 25th percentile
def q1(x):
    return x.quantile(0.25)

# Q2: 50th percentile
def q2(x):
    return x.quantile(0.50)

# Q3: 75th percentile
def q3(x):
    return x.quantile(0.75)


wealth_by_risk_attitude = agents_last_round_df.groupby("risk_attitude", as_index=False)["points"].agg(["mean", "min", "max", q1, q2, q3]) 
wealth_by_risk_attitude

,risk_attitude,mean,min,max,q1,q2,q3
0,0,745.548656,0,3012,279.0,465.0,1110.0
1,1,746.050866,0,3048,279.0,465.0,1112.0
2,2,736.458014,0,3423,279.0,465.0,1107.0
3,3,740.109464,0,3147,279.0,465.0,1104.5
4,4,733.635550,6,3513,279.0,462.0,1083.0
5,5,719.271368,11,2998,285.0,408.5,1083.0
6,6,723.854279,66,3311,301.0,402.0,1088.0
7,7,714.066578,108,2578,282.0,399.0,1089.0
8,8,711.205859,120,2583,280.0,396.0,1088.0
9,9,715.663486,120,2538,285.0,396.5,1089.0
